# Part-1 CODE-BLOCK
- Do not make any changes in the 'CODE-BLOCK'
- Execute all cells in sequence
- Test-Blocks are disabled but for safety do not try to execute any cell marked as Test-Block

In [1]:
# import necessary libraries
import time
import requests
import re
import numpy as np
import pandas as pd
from contextlib import suppress
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Install webdriver manager to automatically detect driver location
service = ChromeService(executable_path=ChromeDriverManager().install())

# Set up ChromeDriver options
chrome_options = Options()
chrome_options.add_argument('--headless') # Run in headless mode to avoid opening a new browser window


In [2]:
def extract_hrefs(n=2, sleep = 15):
    """
    Extracts all hrefs from a paginated web url as a list 
    Args:
        n = number of paginated web urls from which hrefs are to be extracted, default = 5
        sleep = time in seconds to sleep before quitting the driver, default = 15 sec
        url = paginated url from which data is to be extracted
    Returns:
        (list) containining the hrefs found on each page 
    
    """
    
        # Initialize empty list to store results
    all_hrefs = []

    
    # Loop over n list pages
    for i in range(1, n+1):
        # Build URL for current list pages having links to company profiles
        # Don't uncomment following line of code
        url = f"https://wirmarket.wir.ch/de/members/list/?page={i}&resultAmount=100"
        

        # Launch ChromeDriver and load websites
        # In order to disable browser windows opening for websites amend the code of the following line:
        # driver = webdriver.Chrome(service=service, options = chrome_options)
        driver = webdriver.Chrome(service=service)

        # Don't uncomment the code on the following line, it is for testing
        #   driver.implicitly_wait(20) # seconds
        driver.get(url)

        # Wait for all anchor elements to be present
        # WebDriverWait(driver, 180).until(EC.visiblity_of_all_elements_located((By.CSS_SELECTOR, "a")))
        # Due to intermittent functionality of WebDriverWait, we introduce sleep() function to get all anchors
        # This value is presently set at 15 secs but you may change it depending upon internet speed
        time.sleep(sleep)
        # Extract hrefs from all anchor elements on current website
        hrefs = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, "a")]

        # Add hrefs to the list of all hrefs
        all_hrefs.extend(hrefs) # extend flattens the list so that it is no longer a list of lists

        # Close the current browser window
        driver.quit()
    return all_hrefs


In [3]:
### Test Block ! DO NOT EXECUTE
# url = f"https://wirmarket.wir.ch/de/members/list/?page={i}&resultAmount=100"
# all_hrefs_list = extract_hrefs(n=2, sleep = 15, 
#                          url = url)
                          
# # Check the length of all_hrefs list
# # Each list link contains between 400-500 hrefs
# print(len(all_hrefs_list))
# # Check for duplicates
# print(len(set(all_hrefs_list)))

# # Test for function extract_hrefs passed

In [4]:
def unique_urls(urls_list):
    """
    Removes repeated entries of urls in a list which might occur due to use of regex or multiple occurences on a page
    Args:
        urls_list: list containing urls which might be repetitive entries
    Returns: list containing unique urls
    """
    # convert to set
    urls_set = set(urls_list)
    # convert back to list
    unique_urls_list = list(urls_set)
    
    return unique_urls_list


In [5]:
# ## Test-Block !DO NOT EXECUTE
# unique_hrefs = unique_urls(all_hrefs_list)
# print(len(unique_hrefs))

# # Test for function unique_urls passed

In [6]:
def regex_pattern_urls(pattern, hrefs_list): 
    """
    Uses a regex pattern to extract urls of interest out of a list of urls
    pattern: regex pattern used for extracting urls from a list of urls
    hrefs_list: list of hrefs required to be filtered according to the regex pattern
    returns:urls_of_interest a list of urls filtered according to regex pattern provided
        
    """
    # Filter the list 'all_hrefs' to keep only the company profile page urls

    #import regex library
    import re
    # regex pattern for company profile pages
    # Do not uncomment following line of code
#     pattern = r'https:\/\/\w+\.wir\.ch\/de\/companyProfile\/profile\/[0-9A-F]{32}\/info\/\?promo=false$'

    urls_of_interest = []

    # loop over all_hrefs list extracted above with Selenium to extract company profile urls
    for href in hrefs_list:
        match = re.search(pattern, str(href))
        if match:
            url = (match.group())
            urls_of_interest.append(url)
    return urls_of_interest        

In [7]:
# ## Test-Block !DO NOT EXECUTE

# pattern = r'https:\/\/\w+\.wir\.ch\/de\/companyProfile\/profile\/[0-9A-F]{32}\/info\/\?promo=false$'
# company_profile_urls =  regex_pattern_urls(pattern, unique_hrefs)
# print(len(company_profile_urls))
# unique_company_profile_urls = unique_urls(company_profile_urls)
# print(len(unique_company_profile_urls)) # should be 200 (slight variation of up to 5% is acceptable due to variation in internet speed)
# print(unique_company_profile_urls[:10]) #must contain the string "info" in them

# # Test for regex_pattern_urls passed

In [8]:
def extract_info_of_interest(url_list, batch_size = 1000, batch_start_index = 0, sleep=10):
    """
    Extracts all the relevant information from the given unique pages using selenium web driver finders and locators
    Args:
    url_list: list of the pages from where info is to be extracted
    batch_size (int): Number of pages to be processed at a time. It is recommended to process large number of pages in batches of 1000
    batch_start_index(int): index of the url_list from where parsing should start
    sleep(int): seconds to wait for page to load before info can be parsed. It overrides WebDriverWait() function
    Returns:
    list of list of extracted data where each list contains data from a single page
    """
    # Dont't uncomment following line of code, it is for troubleshooting
#   from contextlib import suppress

    # prepare the urls batch to be processed
    batch = url_list[batch_start_index:batch_start_index+batch_size+1]

    company_data = [] # initialize a list to contain lists of all companies' info


    count = 0

    for url in batch:
        company_info = [] # initialize a list to contain one company's info


        # driver control block
        driver = webdriver.Chrome(service=service)
        driver.get(url)
        time.sleep(sleep) # We are using sleep to avoid errors of WebDriverWait though it is timewise costly


    # Using suppress context to ignore known exceptions
    # Do not uncomment following line of code as it may lead to execution errors it is for testing
    #     with suppress(Exception): 

        # Get company names

        try:
    #         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "71-lnk")))

            h3_headings = driver.find_elements(By.CLASS_NAME, "inline-block") # find all elements with class = "inline-block"
            h3_list = []
            for heading in h3_headings: #loop through the list obtained
                h3_list.append(heading.text) # get text of all h3 headings
            company_name = h3_list[0] # company name is the first item in the list
            company_info.append(company_name)   
        except:
            company_info.append('NA')
        finally:
                pass


        # Get company industry

        try:

            uls = driver.find_elements(By.CLASS_NAME, "default")
            list_items = driver.find_elements(By.TAG_NAME,"li")   
            li_list = []
            for li in list_items:
                li_list.append(li.text)
            industry_name = li_list[20] # industry name is the twenty first item in the list
            company_info.append(industry_name)   
        except:
            company_info.append('NA')
        finally:
                pass

        # Get company website if present         
        try:
    #       WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "68-lnk")))
            anchor = driver.find_element(By.ID, "68-lnk")
            website = anchor.get_attribute('href')
            company_info.append(website)
        except:
                company_info.append('NA')
        finally:
                pass

        # Get company telephoone if available
        try:
    #         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "66-lnk")))
            anchor = driver.find_element(By.ID, "66-lnk")
            company_telephone = anchor.get_attribute('href')
            company_info.append(company_telephone)
        except:
            company_info.append('NA')
        finally:
                pass
        # Get company fax if available
        try:
    #         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "fax-lnk")))
            anchor = driver.find_element(By.ID, "fax-lnk")
            company_fax_no = anchor.get_attribute('href')
            company_info.append(company_fax_no)
        except:
            company_info.append('NA')
        finally: 
                pass

       # Get company address and address google map link
        try:
    #       WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "6-lnk")))
            anchor = driver.find_element(By.ID, "6-lnk")
            address = anchor.text
            address_map_link = anchor.get_attribute('href')
            company_info.append(address)
            company_info.append(address_map_link)

        except:
                company_info.append('NA')
        finally:
                pass


        company_data.append(company_info)
        company_info = [] # reinitialize company_info
        
        count+=1
        print("count", count)

        driver.quit()
    return company_data


In [9]:
# ## Test-Block !DO NOT Execute
# company_data_lists = extract_info_of_interest(url_list = unique_company_profile_urls, batch_size = 200, batch_start_index = 0, sleep=10)

In [10]:
# # Test-Block !Do Not Execute
# print(len(company_data_lists)) # should be equal to batch size i.e. 200
# # test for extract_info_of_interest passed

In [11]:
# # Test-Block !DO NOT ExECUTE
# print(len(company_data_lists[-1]))
# print(company_data_lists[:5]) 
# # Should be a list of lists
# # Each list should have seven items
# # First item in each list should be 'company name', second: 'industry', third: website, 
# # fourth: 'tel', fifth 'fax', sixth: 'address', seventh: 'address maplink'
# # data not available should be represented by appropriate remarks

# # test for extract_info_of_interest_passed

In [12]:
def extract_emails_from_homepage(info_list, pattern=r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', index=2):
    """Extracts emails from a list of lists containing a referred website address using a regex pattern
    Args:
    info_list: list of lists containing extracted info including website address
    pattern: regex pattern to cover all possible formats of email addresses
    index(int): index number in the lists where website address is located, default=2
    Returns: info_list_with_emails a list with email addresses appended to info_list
    """

    import requests
    import re
    from bs4 import BeautifulSoup

    info_list_with_emails = info_list.copy()  # make a copy to avoid overwriting original list
    all_links = []
    for page in info_list_with_emails:
        try:
            home_page = str(page[index])
            response = requests.get(home_page)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                hrefs = [link.get('href') for link in soup.find_all('a')]  # get all links on the homepage
                # search for emails in homepage response text
                emails = re.findall(pattern, response.text)
                if emails:
                    unique_emails = set(emails)
                    unique_emails_list = list(unique_emails)
                    page.append(unique_emails_list[0])
                    all_links.append(unique_emails_list[0])
                else:
                    # search for emails in homepage hrefs
                    for href in hrefs:
                        if href and href.startswith (str('mailto:')):
                            emails = [] 
                            email = href.split(':')[1]
                            emails.append(email)
                            page.append(emails[0])
                            all_links.append(email)
                            break
                        else:
                            pass
                        
            else:
                page.append('bad link')
        except:
            page.append('website not accessible')

    print('No of emails found: ', len(all_links))
   
    return info_list_with_emails


In [13]:
def extract_emails_from_allpages(info_list, pattern=r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', index=2):
    """Extracts emails from a list of lists containing a referred website address using a regex pattern
    Args:
    info_list: list of lists containing extracted info including website address
    pattern: regex pattern to cover all possible formats of email addresses
    index(int): index number in the lists where website address is located, default=2
    Returns: info_list_with_emails a list with email addresses appended to info_list
    """

    import requests
    import re
    from bs4 import BeautifulSoup

    info_list_with_emails = info_list.copy()  # make a copy to avoid overwriting original list
    all_links = []
    
    for page in info_list_with_emails:
        try:
            home_page = str(page[index])
            response = requests.get(home_page)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                hrefs = [link.get('href') for link in soup.find_all('a')]  # get all links on the homepage

                # search for emails on the homepage
                emails = re.findall(pattern, response.text)
                if emails:
                    unique_emails = set(emails)
                    unique_emails_list = list(unique_emails)
                    page.append(unique_emails_list[0])
                    all_links.append(unique_emails_list[0])
                else:
                    # search for emails on all links found
                    email_found = False
                    for href in hrefs:
                        if href and href.startswith(str('mailto:')):
                            emails = [] 
                            email = href.split(':')[1]
                            emails.append(email)
                            page.append(emails[0])
                            all_links.append(emails[0])
                            email_found = True
                            break
                        elif 'http' not in href:
                            link = home_page[:-1]+ href
                            link_response = requests.get(link)
                            if link_response.status_code == 200:
                                link_soup = BeautifulSoup(link_response.text, 'html.parser')
                                link_hrefs = [link.get('href') for link in link_soup.find_all('a')]

                                emails = re.findall(pattern, link_response.text)
                                if emails:
                                    unique_emails = set(emails)
                                    unique_emails_list = list(unique_emails)
                                    page.append(unique_emails_list[0])
                                    all_links.append(unique_emails_list[0])
                                    email_found = True
                                    break
                                        
                    if not email_found:
                        page.append("NA")
                        
            else:
                page.append('bad link')
        except:
            page.append('website not accessible')

    print('No of emails found: ', len(all_links), all_links)

   
    return info_list_with_emails


In [14]:
# ## Test_Bloack !DO NOT EXECUTE
# company_data_lists_with_emails = extract_emails_from_websites(company_data_lists)

In [15]:
# ## Test_Block !DO NOT EXECUTE
# print(len(company_data_lists_with_emails)) # should be same as company_data_lists i.e. 200
# print(len(company_data_lists_with_emails[-1])) # should be 8
# print(company_data_lists_with_emails[-5:]) #should have 8 items in each list with emails or remarks added

# # Test for extract_emails_from_websites passed

In [16]:
# ## Test-Block !DO NOT EXECUTE
# # Convert to Dataframe first 100 items
# columns=['company_name', 'industry', 'website', 'telephone', 'fax', 'address', 'address_maplink', 'email']

# df1 = pd.DataFrame(company_data_lists_with_emails[:100], columns=columns)
 

In [17]:
# ## Test-Block !DO NOT EXECUTE
# df1.info()

In [18]:
# ## Test_Block !DO NOT EXECUTE
# df1.head()

In [19]:
# ## Test-Block !DO NOT EXECUTE
# # Convert to DataFrame next 100 items
# columns=['company_name', 'industry', 'website', 'telephone', 'fax', 'address', 'address_maplink', 'email']

# df2 = pd.DataFrame(company_data_lists_with_emails[101:200], columns=columns)


In [20]:
# ## Test-Block !DO NOT EXECUTE
# df2.info()

In [21]:
# ## Test_Block !DO NOT EXECUTE
# df2.head()

In [22]:
# ## Test_Block !DO NOT EXECUTE
# df2.tail()

In [23]:
# # Test-Block !DO nOT EXECUTE
# # Join both Dataframes
# df = pd.concat([df1, df2])

In [24]:
## Test-Block DO NOT EXECUTE
# df.info()

In [25]:
# ## Test_Block !DO NOT EXECUTE
# df.head()==df1.head() # should be same as df1.head()
# # test passed

In [26]:
# ## Test_Block !DO NOT EXECUTE
# df.tail()==df2.tail() # should be same as df2.tail()
# # test passed

### xxxxxxxxxxxxxxxxxxxxxx END OF CODE BLOCKXXXXXXXXXXXXXXXXXXXXXXXX

## XXXXXXXXXXXXX START OF EXECUTION BLOCXXXXXXXXXXXXXX

### Part-I Extract links to all company profile pages

In [27]:
## all_hrefs already extracted. Uncomment and execute only if extracting all_hrefs again
# # extract all_hrefs from 224 list pages on the site 
# start_time = time.time()
# # url = f"https://wirmarket.wir.ch/de/members/list/?page={i}&resultAmount=25"
# all_hrefs = extract_hrefs( n=224, sleep = 17)

# end_time = time.time()
# execution_time = end_time - start_time
# print("execution_time: ", execution_time)


In [28]:
## Test-Block !Uncomment and execute only if extracting all_hrefs again
# print(len(all_hrefs))

In [29]:
# ## all_hrefs.csv already saved. Uncomment and execute only if extracting all_hrefs again
# all_hrefs_dict = {"hrefs": all_hrefs}
# all_hrefs_df = pd.DataFrame(all_hrefs)
# all_hrefs_df.to_csv("all_hrefs.csv", index = False)

In [30]:
# # Duplicates already removed. !Uncomment and execute only if extracting again
# # Remove duplicate or repeated entries from the all_hrefs
# all_hrefs_unique = unique_urls(all_hrefs) 

In [31]:
# # Test-Block !Uncomment and execute only if extracting again
# print(len(all_hrefs_unique))

In [32]:
# ## Already extracted !Uncomment and execute only if extracting again
# # Extract company profile page urls from all unique hrefs
# pattern = r'https:\/\/\w+\.wir\.ch\/de\/companyProfile\/profile\/[0-9A-F]{32}\/info\/\?promo=false$'
# company_profile_urls = regex_pattern_urls(pattern = pattern, hrefs_list = all_hrefs_unique)

In [33]:
# # Test_Block !Uncomment and execute only if extracting again
# print(len(company_profile_urls))

In [34]:
# # Uncomment and execute only if extracting again
# # Remove duplicate or repeated entries from company_profile_urls
# unique_company_profile_urls = unique_urls(company_profile_urls)

#### Checkpoint-1

In [35]:
# ## Test-Block. !Uncomment and execute only if extracting again
# print(len(unique_company_profile_urls)) # should be 14227 (slight variation of up to 5% is acceptable due to variation in internet speed)
# print(unique_company_profile_urls[:10])
# print(unique_company_profile_urls[-10:])

In [36]:
# # company_profile_url already saved as profile_urls.csv. Uncomment and execute only if extracting again
# # Save as csv
# company_profile_url_dict = {"urls": unique_company_profile_urls}
# company_profile_url_df = pd.DataFrame(company_profile_url_dict)
# company_profile_url_df.to_csv('profile_urls.csv', index = False)

In [37]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
# import profile_urls.csv as dataframe
unique_company_profiles_df = pd.read_csv("profile_urls.csv")

In [38]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
unique_company_profiles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14227 entries, 0 to 14226
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   urls    14227 non-null  object
dtypes: object(1)
memory usage: 111.3+ KB


In [39]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
unique_company_profile_urls = unique_company_profiles_df['urls'].tolist()


In [40]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
print(len(unique_company_profile_urls))

14227


In [41]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
print(unique_company_profile_urls[-5:])

['https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF26282A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF3C152A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/58B85543DFA09858E05400144FF95A47/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF27E62A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF55272A65E0540010E0244DC9/info/?promo=false']


### xxxxxxxxxxxxxxxxxxx END OF PART-1 xxxxxxxxxxxxxxxxxxxxxxxxxxx

## Part-2 Batch Processing

### Batch-1, batch_size = 1000, batch_start_index = 0

#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [42]:
# # Extract info of interst
# start_time = time.time()
# batch1_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 0, sleep=10)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### checkpoint-1

In [43]:
# # Test-Block ! Execute only if you run the batch again
# print(len(batch1_initial)) # Should be equal to batch_size + 1
# print(len(batch1_initial[-1])) # Should be equal to 7

In [44]:
## Test_Block ! Execute only if you run the batch again
# print(batch1_initial[:3])

### End of checkpoint-1

In [45]:
batch1df = pd.read_csv('batch1.csv')

In [46]:
batch1df_initial = batch1df.drop(['email'], axis = 1)


In [47]:
batch1df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          578 non-null    object
 3   telephone        864 non-null    object
 4   fax              572 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [48]:
batch1_initial = batch1df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen']]


#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [49]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch1 = extract_emails_from_homepage(batch1_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  340
execution time:  1030.6990110874176


### checkpoint-2

In [50]:
print(len(batch1)) # should be same as batch_size i.e. 1001
print(len(batch1[-1])) # should be 8
print(batch1[-5:]) #should have 8 items in each list with emails or remarks added


1001
8
[['STB Holzbau GmbH', 'Einbau von Fenster, Türen und Innenausbau, Einbauküchen, Einbaumöbel', nan, 'tel:055 444 31 68', 'fax:055 444 31 68', 'Haslenstrasse 28b\n8862 Schübelbach', 'https://www.google.com/maps/place/Haslenstrasse%2028b+8862+Sch%C3%BCbelbach', 'website not accessible'], ['Beauty Solar Sonnenland GmbH', 'Saunas, Solarien', nan, 'tel:041 280 15 15', 'fax:041 631 08 07', 'Schulhausstrasse 4\n6052 Hergiswil NW', 'https://www.google.com/maps/place/Schulhausstrasse%204+6052+Hergiswil%20NW', 'website not accessible'], ['Elektro-Brizzi AG', 'Elektroinstallation', nan, 'tel:052 386 22 22', 'fax:052 386 22 93', 'Heinrich Gujer-Strasse 5\n8494 Bauma', 'https://www.google.com/maps/place/Heinrich%20Gujer-Strasse%205+8494+Bauma', 'website not accessible'], ['Keller Wärme & Wasser AG', 'Sanitär- und Heizungsinstallation', nan, 'tel:032 373 51 53', 'fax:032 373 27 71', 'Erlenstrasse 15\n2555 Brügg BE', 'https://www.google.com/maps/place/Erlenstrasse%2015+2555+Br%C3%BCgg%20BE', 'w

### End of checkpoint-2

In [51]:
# Columns to be used for all batches
columns=['company_name', 'industry', 'website', 'telephone', 'fax', 'address', 'address_maplink', 'email']


In [52]:
# # Convert to Dataframe batch1
df1 = pd.DataFrame(batch1, columns=columns)


### Checkpoint-3

In [53]:
# # Test-Block ! Execute only if you run the batch again
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          578 non-null    object
 3   telephone        864 non-null    object
 4   fax              572 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
 7   email            868 non-null    object
dtypes: object(8)
memory usage: 62.7+ KB


In [54]:
df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,8c4075d5481d476e945486754f783364@sentry.io
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,info@nicolediem.ch


### End of Checkpoint-3

In [55]:
# Save as csv file
df1.to_csv('batch1.csv', index = False)

### xxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-2, batch_size = 1000, batch_start_index = 1001

#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [56]:
# # Extract info of interst
# start_time = time.time()
# batch2_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 1001, sleep=10)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### checkpoint-1

In [57]:
# ## Test_Block ! Execute only if you run the batch again
# print(len(batch2_initial))
# print(len(batch2_initial[-1]))

In [58]:
# ## Test_Block ! Execute only if you run the batch again
# print(batch2_initial[:3])

### End of Checkpoint-1

In [59]:
batch2df = pd.read_csv('batch2.csv')

In [60]:
batch2df_initial = batch2df.drop(['email'], axis = 1)

In [61]:
batch2df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     987 non-null    object
 1   industry         988 non-null    object
 2   website          578 non-null    object
 3   telephone        867 non-null    object
 4   fax              559 non-null    object
 5   address          987 non-null    object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [62]:
batch2_initial = batch2df_initial.values.tolist()
print(batch2_initial[:3])

[['Modehaus Peter', 'Detailhandel mit Herrenbekleidung', nan, 'tel:041 970 17 57', 'fax:041 970 07 57', 'Hauptgasse 36\n6130 Willisau', 'https://www.google.com/maps/place/Hauptgasse%2036+6130+Willisau'], ['VB ORGANISATION Sàrl', 'Offsetdruck', 'http://www.imprimerievb.ch/', 'tel:079 205 44 53', nan, 'chemin St-Hubert 32\n1950 Sion', 'https://www.google.com/maps/place/chemin%20St-Hubert%2032+1950+Sion'], ['Heinz Stucki', 'Anbau von Getreide (ohne Reis), Hülsenfrüchten und Ölsaaten', nan, 'tel:031 781 05 03', 'fax:031 781 05 03', 'Hubmatt\n3116 Noflen BE', 'https://www.google.com/maps/place/Hubmatt+3116+Noflen%20BE']]


#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [63]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch2 = extract_emails_from_homepage(batch2_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 15 sec per page)
#### Entire Batch can take upto 5 hours - Uncomment (Ctrl + A followed by Ctrl + / will uncomment the code.
#### Execute(Shift + Enter) only if needed

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  341
execution time:  920.9898567199707


### Checkpoint-2

In [64]:
print(len(batch2)) # should be same as batch_size i.e. 1000
print(len(batch2[-1])) # should be 8
print(batch2[-5:]) #should have 8 items in each list with emails or remarks added

1001
7
[['MBA AG', 'Grosshandel mit Bergwerks-, Bau- und Baustoffmaschinen', 'http://www.mba-baumaschinen.ch/', 'tel:044 838 61 11', nan, 'Zürichstrasse 50\n8303 Bassersdorf', 'https://www.google.com/maps/place/Z%C3%BCrichstrasse%2050+8303+Bassersdorf', 'info@mba-maschinen.com'], ['Mächler GU AG', 'Garten- und Landschaftsbau sowie Erbringung von sonstigen gärtnerischen Dienstleistungen', 'http://www.maechler-gu.ch/', 'tel:055 451 11 31', 'fax:055 451 11 34', 'St.Gallerstrasse 58\n8853 Lachen SZ', 'https://www.google.com/maps/place/St.Gallerstrasse%2058+8853+Lachen%20SZ', 'info@maechler-gu.ch'], ['Garasch 106 AG', 'lnstandhaltung und Reparatur von Automobilen', 'https://garasch106.ch/', 'tel:041 624 40 24', nan, 'Stanserstrasse 106\n6373 Ennetbürgen', 'https://www.google.com/maps/place/Stanserstrasse%20106+6373+Ennetb%C3%BCrgen', 'auto@garasch106.ch'], ['Bumbachsäge AG', 'Sägewerke', 'http://www.bumbachsaege.ch/', 'tel:034 493 35 57', 'fax:034 493 31 84', 'Bumbachsäge 172\n6197 Schangna

### End of Checkpoint-2

In [65]:
# # Convert to Dataframe batch1
df2 = pd.DataFrame(batch2, columns=columns)


### Checkpoint 3

In [66]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     987 non-null    object
 1   industry         988 non-null    object
 2   website          578 non-null    object
 3   telephone        867 non-null    object
 4   fax              559 non-null    object
 5   address          987 non-null    object
 6   address_maplink  1001 non-null   object
 7   email            865 non-null    object
dtypes: object(8)
memory usage: 62.7+ KB


In [67]:
df2.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Modehaus Peter,Detailhandel mit Herrenbekleidung,NaN,tel:041 970 17 57,fax:041 970 07 57,Hauptgasse 36\n6130 Willisau,https://www.google.com/maps/place/Hauptgasse%2...,website not accessible
1,VB ORGANISATION Sàrl,Offsetdruck,http://www.imprimerievb.ch/,tel:079 205 44 53,NaN,chemin St-Hubert 32\n1950 Sion,https://www.google.com/maps/place/chemin%20St-...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Heinz Stucki,"Anbau von Getreide (ohne Reis), Hülsenfrüchten...",NaN,tel:031 781 05 03,fax:031 781 05 03,Hubmatt\n3116 Noflen BE,https://www.google.com/maps/place/Hubmatt+3116...,website not accessible
3,Heizplan HPA AG,Sanitär- und Heizungsinstallation,http://www.heizplan.ch/,tel:081 750 34 50,fax:081 750 34 59,Karmaad 36\n9473 Gams,https://www.google.com/maps/place/Karmaad%2036...,kontakt@heizplan.ch
4,Hoppler Tiefbohrungen GmbH,Test- und Suchbohrung,http://www.hoppler-gmbh.ch/,tel:056 634 40 46,fax:056 634 40 75,Allmendstrasse 1\n5621 Zufikon,https://www.google.com/maps/place/Allmendstras...,info@hoppler-gmbh.ch


### End of Checkpoint-3

In [68]:
# save as csv
df2.to_csv('batch2.csv', index = False)

In [69]:
# Merge the two dataframes
df = pd.concat([df1, df2])

### Checkpoint-4

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2002 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1988 non-null   object
 1   industry         1989 non-null   object
 2   website          1156 non-null   object
 3   telephone        1731 non-null   object
 4   fax              1131 non-null   object
 5   address          1988 non-null   object
 6   address_maplink  2002 non-null   object
 7   email            1733 non-null   object
dtypes: object(8)
memory usage: 140.8+ KB


In [71]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,8c4075d5481d476e945486754f783364@sentry.io
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,info@nicolediem.ch


In [72]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,MBA AG,"Grosshandel mit Bergwerks-, Bau- und Baustoffm...",http://www.mba-baumaschinen.ch/,tel:044 838 61 11,NaN,Zürichstrasse 50\n8303 Bassersdorf,https://www.google.com/maps/place/Z%C3%BCrichs...,info@mba-maschinen.com
997,Mächler GU AG,Garten- und Landschaftsbau sowie Erbringung vo...,http://www.maechler-gu.ch/,tel:055 451 11 31,fax:055 451 11 34,St.Gallerstrasse 58\n8853 Lachen SZ,https://www.google.com/maps/place/St.Gallerstr...,info@maechler-gu.ch
998,Garasch 106 AG,lnstandhaltung und Reparatur von Automobilen,https://garasch106.ch/,tel:041 624 40 24,NaN,Stanserstrasse 106\n6373 Ennetbürgen,https://www.google.com/maps/place/Stanserstras...,auto@garasch106.ch
999,Bumbachsäge AG,Sägewerke,http://www.bumbachsaege.ch/,tel:034 493 35 57,fax:034 493 31 84,Bumbachsäge 172\n6197 Schangnau,https://www.google.com/maps/place/Bumbachs%C3%...,info@bumbachsaege.ch
1000,MEUBLES DESCARTES SA,Inserate,http://www.decarte.ch/,tel:027 743 43 43,fax:027 743 43 44,Route du Léman 33\n1907 Saxon,https://www.google.com/maps/place/Route%20du%2...,None


In [73]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [74]:
df.tail() == df2.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,True,True,True,True,False,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,True,True,True,True,True,True
1000,True,True,True,True,True,True,True,False


### End of Checkpoint-4

### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-3, batch_size = 1000, batch_start_index = 2001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [75]:
# # Extract info of interst
# start_time = time.time()
# batch3_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 2001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### checkpoint-1

In [76]:
# # Test-Block ! Execute only if running the batch again
# print(len(batch3_initial))
# print(len(batch3_initial[-1]))

In [77]:
# # Test-Block ! Execute only if running the batch again
# print(batch3_initial[:3])

### End of Checkpoint-1

In [78]:
batch3df = pd.read_csv('batch3.csv')

In [79]:
batch3df_initial = batch3df.drop(['email'], axis = 1)

In [80]:
batch3df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     986 non-null    object
 1   industry         985 non-null    object
 2   website          561 non-null    object
 3   telephone        880 non-null    object
 4   fax              548 non-null    object
 5   address          986 non-null    object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [81]:
batch3_initial = batch3df_initial.values.tolist()
print(batch3_initial[:3])

[['MEUBLES DESCARTES SA', 'Inserate', 'http://www.decarte.ch/', 'tel:027 743 43 43', 'fax:027 743 43 44', 'Route du Léman 33\n1907 Saxon', 'https://www.google.com/maps/place/Route%20du%20L%C3%A9man%2033+1907+Saxon'], ['Eticolle Schoechli SA', 'Sonstiges Drucken a. n. g.', 'http://www.eticolle.ch/', 'tel:027 452 25 26', nan, 'Techno-pôle 2\n3960 Sierre', 'https://www.google.com/maps/place/Techno-p%C3%B4le%202+3960+Sierre'], ['Ettima AG', 'Grosshandel mit Werkzeugmaschinen', 'http://www.ettima.ch/', 'tel:031 819 56 26', 'fax:031 819 56 63', 'Bernstrasse 25\n3125 Toffen', 'https://www.google.com/maps/place/Bernstrasse%2025+3125+Toffen']]


### End of Checkpoint-1

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [82]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch3 = extract_emails_from_homepage(batch3_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  312
execution time:  934.1918461322784


### Checkpoint-2

In [83]:
print(len(batch3)) # should be same as batch_size i.e. 1000
print(len(batch3[-1])) # should be 8
print(batch3[-5:]) #should have 8 items in each list with emails or remarks added

1001
7
[['Hirt Schreinerei GmbH', 'Schreinerarbeiten im Innenausbau', 'http://www.hirt-schreinerei.ch/', 'tel:061 901 55 88', 'fax:061 901 83 04', 'Bächliackerstrasse 4\n4402 Frenkendorf', 'https://www.google.com/maps/place/B%C3%A4chliackerstrasse%204+4402+Frenkendorf', 'bad link'], ['René Bührer AG', 'Detailhandel mit Vorhängen, Teppichen, Fussbodenbelägen und Tapeten', nan, 'tel:052 672 16 30', 'fax:052 672 16 40', 'Rheinfallstrasse 7\n8212 Neuhausen am Rheinfall', 'https://www.google.com/maps/place/Rheinfallstrasse%207+8212+Neuhausen%20am%20Rheinfall', 'website not accessible'], ['EFOS Flugschule GmbH', 'Fahr- und Flugschulen', 'http://www.efos.ch/', 'tel:044 861 07 04', nan, 'Flughafenstrasse 14\n8302 Kloten', 'https://www.google.com/maps/place/Flughafenstrasse%2014+8302+Kloten', '20ws@efos.ch'], ['Hess Druck AG', 'Offsetdruck', 'http://www.hessdruck.ch/', 'tel:071 658 61 80', nan, 'Schützlerweg 3\n8560 Märstetten', 'https://www.google.com/maps/place/Sch%C3%BCtzlerweg%203+8560+M%C3

### End of Checkpoint-2

In [84]:
# # Convert to Dataframe batch3
df3 = pd.DataFrame(batch3, columns=columns)


### Checkpoint 3

In [85]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     986 non-null    object
 1   industry         985 non-null    object
 2   website          561 non-null    object
 3   telephone        880 non-null    object
 4   fax              548 non-null    object
 5   address          986 non-null    object
 6   address_maplink  1001 non-null   object
 7   email            850 non-null    object
dtypes: object(8)
memory usage: 62.7+ KB


In [86]:
df3.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,MEUBLES DESCARTES SA,Inserate,http://www.decarte.ch/,tel:027 743 43 43,fax:027 743 43 44,Route du Léman 33\n1907 Saxon,https://www.google.com/maps/place/Route%20du%2...,None
1,Eticolle Schoechli SA,Sonstiges Drucken a. n. g.,http://www.eticolle.ch/,tel:027 452 25 26,NaN,Techno-pôle 2\n3960 Sierre,https://www.google.com/maps/place/Techno-p%C3%...,info@eticolle.ch
2,Ettima AG,Grosshandel mit Werkzeugmaschinen,http://www.ettima.ch/,tel:031 819 56 26,fax:031 819 56 63,Bernstrasse 25\n3125 Toffen,https://www.google.com/maps/place/Bernstrasse%...,None
3,Alushi AG,Spezielle Reinigung von Gebäuden und Reinigung...,http://www.alushi.ch/,tel:052 376 16 80,fax:052 376 16 87,Frauenfelderstrasse 66\n9548 Matzingen,https://www.google.com/maps/place/Frauenfelder...,None
4,Studer Optik,Detailhandel mit Brillen und anderen Sehhilfen,http://www.studeroptik.ch/,tel:032 392 25 15,fax:032 392 28 89,Stadtplatz 60\n3270 Aarberg,https://www.google.com/maps/place/Stadtplatz%2...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...


### End of Checkpoint-3

In [87]:
#save as csv
df3.to_csv('batch3.csv', index = False)

In [88]:
# Merge the two dataframes
df = pd.concat([df, df3])

### Checkpoint-4

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3003 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     2974 non-null   object
 1   industry         2974 non-null   object
 2   website          1717 non-null   object
 3   telephone        2611 non-null   object
 4   fax              1679 non-null   object
 5   address          2974 non-null   object
 6   address_maplink  3003 non-null   object
 7   email            2583 non-null   object
dtypes: object(8)
memory usage: 211.1+ KB


In [90]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,8c4075d5481d476e945486754f783364@sentry.io
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,info@nicolediem.ch


In [91]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,Hirt Schreinerei GmbH,Schreinerarbeiten im Innenausbau,http://www.hirt-schreinerei.ch/,tel:061 901 55 88,fax:061 901 83 04,Bächliackerstrasse 4\n4402 Frenkendorf,https://www.google.com/maps/place/B%C3%A4chlia...,bad link
997,René Bührer AG,"Detailhandel mit Vorhängen, Teppichen, Fussbod...",NaN,tel:052 672 16 30,fax:052 672 16 40,Rheinfallstrasse 7\n8212 Neuhausen am Rheinfall,https://www.google.com/maps/place/Rheinfallstr...,website not accessible
998,EFOS Flugschule GmbH,Fahr- und Flugschulen,http://www.efos.ch/,tel:044 861 07 04,NaN,Flughafenstrasse 14\n8302 Kloten,https://www.google.com/maps/place/Flughafenstr...,20ws@efos.ch
999,Hess Druck AG,Offsetdruck,http://www.hessdruck.ch/,tel:071 658 61 80,NaN,Schützlerweg 3\n8560 Märstetten,https://www.google.com/maps/place/Sch%C3%BCtzl...,info@hessdruck.ch
1000,Rhomberg Schmuck AG,Detailhandel mit Uhren und Schmuck,http://www.schmuck.ch/,tel:071 310 15 80,NaN,Zürcher-Strasse\n9015 St. Gallen,https://www.google.com/maps/place/Z%C3%BCrcher...,None


In [92]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [93]:
df.tail() == df3.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,True,True,True,True,True,True,True,True
997,True,True,False,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,True,True,False,True,True,True
1000,True,True,True,True,False,True,True,False


### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-4, batch_size = 1000, batch_start_index = 3001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [94]:
# # Extract info of interst
# start_time = time.time()
# batch4_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 3001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### checkpoint-1

In [95]:
# # Test-Block ! Uncomment and execute only if batch is run again
# print(len(batch4_initial))
# print(len(batch4_initial[-1]))

In [96]:
# # Test-Block ! Uncomment and execute only if batch is run again
# print(batch4_initial[:3])

### End of Checkpoint-1

In [97]:
batch4df = pd.read_csv('batch4.csv')

In [98]:
batch4df_initial = batch4df.drop(['email'], axis = 1)

In [99]:
batch4df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          563 non-null    object
 3   telephone        883 non-null    object
 4   fax              530 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [100]:
batch4_initial = batch4df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern', 'info@ringler-strahm.ch'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans', '8c4075d5481d476e945486754f783364@sentry.io'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg']]


#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [101]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch4 = extract_emails_from_homepage(batch4_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

No of emails found:  349
execution time:  1157.1034061908722


### Checkpoint-2

In [102]:
print(len(batch4)) # should be same as batch_size i.e. 1000
print(len(batch4[-1])) # should be 8
print(batch4[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Martin Stoller Transporte', 'Güterbeförderung im Strassenverkehr', nan, 'tel:031 741 24 15', nan, 'Weidstrasse 2\n3184 Wünnewil', 'https://www.google.com/maps/place/Weidstrasse%202+3184+W%C3%BCnnewil', 'website not accessible'], ['Christoph Hasler Schreinerei', 'Schreinerarbeiten im Innenausbau', 'http://www.hasler-schreinerei.ch/', 'tel:071 917 11 55', 'fax:071 917 18 48', 'Käsereistrasse 1\n9555 Tobel', 'https://www.google.com/maps/place/K%C3%A4sereistrasse%201+9555+Tobel', 'info@hasler-schreinerei.ch'], ['MoRailSo AG', 'Grosshandel mit sonstigen Maschinen und Ausrüstungen', 'https://www.ramorail.com/', 'tel:079 463 97 50', nan, 'Rothmatte 7\n6022 Grosswangen', 'https://www.google.com/maps/place/Rothmatte%207+6022+Grosswangen', 'bad link'], ['Elektro Schuler AG', 'Elektroinstallation', nan, 'tel:041 631 03 31', 'fax:041 631 03 32', 'Schulhausstrasse 4\n6052 Hergiswil NW', 'https://www.google.com/maps/place/Schulhausstrasse%204+6052+Hergiswil%20NW', 'website not accessible']

### End of Checkpoint-2

In [103]:
# # Convert to Dataframe batch1
df4 = pd.DataFrame(batch4, columns=columns)


### Checkpoint 3

In [104]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          563 non-null    object
 3   telephone        883 non-null    object
 4   fax              530 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
 7   email            871 non-null    object
dtypes: object(8)
memory usage: 62.7+ KB


In [105]:
df4.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Rhomberg Schmuck AG,Detailhandel mit Uhren und Schmuck,http://www.schmuck.ch/,tel:071 310 15 80,NaN,Zürcher-Strasse\n9015 St. Gallen,https://www.google.com/maps/place/Z%C3%BCrcher...,None
1,GanzImmo AG,"Verwaltung von Grundstücken, Gebäuden und Wohn...",http://www.ganzimmo.ch/,tel:052 213 56 65,fax:052 213 31 74,Schaffhauserstrasse 79\n8401 Winterthur,https://www.google.com/maps/place/Schaffhauser...,None
2,Coiffure Daniel Moll,Artikel im Shop,http://www.coiffeur-kriens.com/,tel:041 310 05 41,NaN,Amlehnstrasse 54\n6010 Kriens,https://www.google.com/maps/place/Amlehnstrass...,8c4075d5481d476e945486754f783364@sentry.io
3,Rondell AG,"Kauf und Verkauf von eigenen Grundstücken, Geb...",NaN,tel:041 320 53 73,NaN,Weinhalde 12\n6010 Kriens,https://www.google.com/maps/place/Weinhalde%20...,website not accessible
4,Rémy Bühler Malergeschäft Rothrist,Malerei,http://www.maler-buehler.ch/,tel:062 794 44 64,NaN,Pfaffernweg 7\n4852 Rothrist,https://www.google.com/maps/place/Pfaffernweg%...,cid_image001_png@01D443CD.png


### End of Checkpoint-3

In [106]:
#save as csv
df4.to_csv('batch4.csv', index = False)

In [107]:
# Merge the two dataframes
df = pd.concat([df, df4])

### Checkpoint-4

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4004 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     3975 non-null   object
 1   industry         3975 non-null   object
 2   website          2280 non-null   object
 3   telephone        3494 non-null   object
 4   fax              2209 non-null   object
 5   address          3975 non-null   object
 6   address_maplink  4004 non-null   object
 7   email            3454 non-null   object
dtypes: object(8)
memory usage: 281.5+ KB


In [109]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,8c4075d5481d476e945486754f783364@sentry.io
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,info@nicolediem.ch


In [110]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,Martin Stoller Transporte,Güterbeförderung im Strassenverkehr,NaN,tel:031 741 24 15,NaN,Weidstrasse 2\n3184 Wünnewil,https://www.google.com/maps/place/Weidstrasse%...,website not accessible
997,Christoph Hasler Schreinerei,Schreinerarbeiten im Innenausbau,http://www.hasler-schreinerei.ch/,tel:071 917 11 55,fax:071 917 18 48,Käsereistrasse 1\n9555 Tobel,https://www.google.com/maps/place/K%C3%A4serei...,info@hasler-schreinerei.ch
998,MoRailSo AG,Grosshandel mit sonstigen Maschinen und Ausrüs...,https://www.ramorail.com/,tel:079 463 97 50,NaN,Rothmatte 7\n6022 Grosswangen,https://www.google.com/maps/place/Rothmatte%20...,bad link
999,Elektro Schuler AG,Elektroinstallation,NaN,tel:041 631 03 31,fax:041 631 03 32,Schulhausstrasse 4\n6052 Hergiswil NW,https://www.google.com/maps/place/Schulhausstr...,website not accessible
1000,Camping Lac des Brenets,Inserate,http://www.camping-brenets.ch/,tel:032 932 16 18,fax:032 932 16 39,2416 Les Brenets,https://www.google.com/maps/place/undefined+24...,info@camping-brenets.ch


In [111]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [112]:
df.tail() == df4.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,True,True,False,True,False,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,False,True,True,True,True,True
1000,True,True,True,True,True,True,True,True


### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-5, batch_size = 1000, batch_start_index = 4001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [113]:
# # Extract info of interst
# start_time = time.time()
# batch5_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 4001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### checkpoint-1

In [114]:
# # Test-Block !Uncomment and execute only if the batch is run again
# print(len(batch5_initial))
# print(len(batch5_initial[-1]))

In [115]:
# # Test-Block !Uncomment and execute only if the batch is run again
# print(batch5_initial[:3])

### End of Checkpoint-1

In [116]:
batch5df = pd.read_csv('batch5.csv')

In [117]:
batch5df_initial = batch5df.drop(['email'], axis = 1)

In [118]:
batch5df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          556 non-null    object
 3   telephone        893 non-null    object
 4   fax              570 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [119]:
batch5_initial = batch5df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern', 'info@ringler-strahm.ch'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans', '8c4075d5481d476e945486754f783364@sentry.io'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg']]


#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [120]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch5 = extract_emails_from_homepage(batch5_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

No of emails found:  325
execution time:  938.4168407917023


### Checkpoint-2

In [121]:
print(len(batch5)) # should be same as batch_size i.e. 1000
print(len(batch5[-1])) # should be 8
print(batch5[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Vinolatino GmbH', 'Grosshandel mit Wein und Spirituosen', nan, nan, nan, 'Hauptstrasse 33\n6280 Hochdorf', 'https://www.google.com/maps/place/Hauptstrasse%2033+6280+Hochdorf', 'website not accessible'], ['Cometal SA', 'Herstellung von Metallkonstruktionen', 'http://www.cometal.ch/', 'tel:091 859 10 22', 'fax:091 859 27 03', 'via Cantonale\n6595 Riazzino', 'https://www.google.com/maps/place/via%20Cantonale+6595+Riazzino'], ['HCDM Beratungen GmbH', 'Unternehmensberatung', nan, 'tel:079 445 69 40', nan, 'Mühleweg 7\n7304 Maienfeld', 'https://www.google.com/maps/place/M%C3%BChleweg%207+7304+Maienfeld', 'website not accessible'], ['Eigenheer Elektro AG', 'Elektroinstallation', 'http://www.eigenheer-elektro.ch/', 'tel:052 317 13 79', nan, 'Burgstrasse 5\n8450 Andelfingen', 'https://www.google.com/maps/place/Burgstrasse%205+8450+Andelfingen', 'info@eigenheer-elektro.ch'], ['Papeterie Zumstein AG', 'Detailhandel mit Schreibwaren und Bürobedarf', nan, 'tel:044 211 77 70', nan, 'Rennwe

### End of Checkpoint-2

In [122]:
# # Convert to Dataframe batch1
df5 = pd.DataFrame(batch5, columns=columns)


### Checkpoint 3

In [123]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          556 non-null    object
 3   telephone        893 non-null    object
 4   fax              570 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
 7   email            861 non-null    object
dtypes: object(8)
memory usage: 62.7+ KB


In [124]:
df5.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Camping Lac des Brenets,Inserate,http://www.camping-brenets.ch/,tel:032 932 16 18,fax:032 932 16 39,2416 Les Brenets,https://www.google.com/maps/place/undefined+24...,info@camping-brenets.ch
1,Elektro Capaul AG,Elektroinstallation,NaN,tel:081 511 20 22,fax:081 511 20 21,Promenada 3\n7018 Flims Waldhaus,https://www.google.com/maps/place/Promenada%20...,website not accessible
2,Parkhotel Bellevue & Spa,"Hotels, Gasthöfe und Pensionen mit Restaurant",http://www.parkhotel-bellevue.ch/,tel:033 673 80 00,fax:033 673 80 01,Bellevuestrasse 15\n3715 Adelboden,https://www.google.com/maps/place/Bellevuestra...,INFO@BELLEVUE-PARKHOTEL.CH
3,Supersaxo Gerold,Malerei und Gipserei ohne ausgeprägten Schwerp...,NaN,tel:g.supersaxo@valaiscom.ch,fax:027 957 45 16,Haus Rio\n3906 Saas-Fee,https://www.google.com/maps/place/Haus%20Rio+3...,website not accessible
4,Jenni Baumaschinen AG,"Grosshandel mit Bergwerks-, Bau- und Baustoffm...",http://www.rammax.ch/Joomla/,tel:041 920 36 62,fax:041 920 35 34,Haldenmattstrasse 2\n6210 Sursee,https://www.google.com/maps/place/Haldenmattst...,bad link


### End of Checkpoint-3

In [125]:
# save as csv
df5.to_csv('batch5.csv', index = False)

In [126]:
#### Merge the two dataframes
df = pd.concat([df, df5])

### Checkpoint-4

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5005 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     4976 non-null   object
 1   industry         4976 non-null   object
 2   website          2836 non-null   object
 3   telephone        4387 non-null   object
 4   fax              2779 non-null   object
 5   address          4976 non-null   object
 6   address_maplink  5005 non-null   object
 7   email            4315 non-null   object
dtypes: object(8)
memory usage: 351.9+ KB


In [128]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,8c4075d5481d476e945486754f783364@sentry.io
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,info@nicolediem.ch


In [129]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,Vinolatino GmbH,Grosshandel mit Wein und Spirituosen,NaN,NaN,NaN,Hauptstrasse 33\n6280 Hochdorf,https://www.google.com/maps/place/Hauptstrasse...,website not accessible
997,Cometal SA,Herstellung von Metallkonstruktionen,http://www.cometal.ch/,tel:091 859 10 22,fax:091 859 27 03,via Cantonale\n6595 Riazzino,https://www.google.com/maps/place/via%20Canton...,None
998,HCDM Beratungen GmbH,Unternehmensberatung,NaN,tel:079 445 69 40,NaN,Mühleweg 7\n7304 Maienfeld,https://www.google.com/maps/place/M%C3%BChlewe...,website not accessible
999,Eigenheer Elektro AG,Elektroinstallation,http://www.eigenheer-elektro.ch/,tel:052 317 13 79,NaN,Burgstrasse 5\n8450 Andelfingen,https://www.google.com/maps/place/Burgstrasse%...,info@eigenheer-elektro.ch
1000,Papeterie Zumstein AG,Detailhandel mit Schreibwaren und Bürobedarf,NaN,tel:044 211 77 70,NaN,Rennweg 19\n8001 Zürich,https://www.google.com/maps/place/Rennweg%2019...,website not accessible


In [130]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [131]:
df.tail() == df5.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,True,True,False,False,False,True,True,True
997,True,True,True,True,True,True,True,False
998,True,True,False,True,False,True,True,True
999,True,True,True,True,False,True,True,True
1000,True,True,False,True,False,True,True,True


### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-6, batch_size = 1000, batch_start_index = 5001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [132]:
# # Extract info of interst
# start_time = time.time()
# batch6_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 5001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### checkpoint-1

In [133]:
# # Test-Block !Uncomment and execute only if batch is run again
# print(len(batch6_initial))
# print(len(batch6_initial[-1]))

In [134]:
# # Test-Block !Uncomment and execute only if batch is run again
# print(batch6_initial[:3])

### End of Checkpoint-1

In [135]:
batch6df = pd.read_csv('batch6.csv')

In [136]:
batch6df_initial = batch6df.drop(['email'], axis = 1)

In [137]:
batch6df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     999 non-null    object
 1   industry         1000 non-null   object
 2   website          575 non-null    object
 3   telephone        891 non-null    object
 4   fax              570 non-null    object
 5   address          999 non-null    object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [138]:
batch6_initial = batch6df_initial.values.tolist()
print(batch6_initial[:3])

[['Papeterie Zumstein AG', 'Detailhandel mit Schreibwaren und Bürobedarf', nan, 'tel:044 211 77 70', nan, 'Rennweg 19\n8001 Zürich', 'https://www.google.com/maps/place/Rennweg%2019+8001+Z%C3%BCrich'], ['Elektro Schuler AG', 'Elektroinstallation', nan, 'tel:041 612 06 33', nan, 'Forellenhof/Stanserstr.\n6362 Stansstad', 'https://www.google.com/maps/place/Forellenhof/Stanserstr.+6362+Stansstad'], ['Rüedi Büromaschinen', 'Detailhandel mit Telekommunikationsgeräten', 'http://www.rueedi-bueromaschinen.ch/', 'tel:034 415 19 23', 'fax:034 415 24 80', 'Gässli 2\n3473 Alchenstorf', 'https://www.google.com/maps/place/G%C3%A4ssli%202+3473+Alchenstorf']]


#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [139]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch6 = extract_emails_from_homepage(batch6_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  351
execution time:  1168.1427521705627


### Checkpoint-2

In [140]:
print(len(batch6)) # should be same as batch_size i.e. 1000
print(len(batch6[-1])) # should be 8
print(batch6[-5:]) #should have 8 items in each list with emails or remarks added

1001
7
[['Max Auer', 'Vermietung und Verpachtung von eigenen oder geleasten Grundstücken', nan, 'tel:071 911 21 42', 'fax:071 911 85 27', 'Agathafeld 22\n9512 Rossrüti', 'https://www.google.com/maps/place/Agathafeld%2022+9512+Rossr%C3%BCti', 'website not accessible'], ['Rino Weder AG', 'Mechanische Werkstätten', 'http://www.rinoweder.ch/', 'tel:071 763 60 50', 'fax:071 763 60 51', 'Roggenweg 1\n9463 Oberriet SG', 'https://www.google.com/maps/place/Roggenweg%201+9463+Oberriet%20SG', 'verkauf@rinoweder.ch'], ['Garage Andermatt AG Baar', 'lnstandhaltung und Reparatur von Automobilen', 'http://www.garage-andermatt.ch/', 'tel:041 760 46 46', nan, 'Ruessenstrasse 22\n6340 Baar', 'https://www.google.com/maps/place/Ruessenstrasse%2022+6340+Baar', 'info@garage-andermatt.ch'], ['Pizzeria Mare Monte, Ekrem Sinani', 'Restaurants, Imbissstuben, Tea-Rooms und Gelaterias', nan, 'tel:071 845 60 06', 'fax:071 845 60 07', 'Blumenstrasse 8\n9403 Goldach', 'https://www.google.com/maps/place/Blumenstrasse%

### End of Checkpoint-2

In [141]:
# # Convert to Dataframe batch1
df6 = pd.DataFrame(batch6, columns=columns)


In [142]:
# # Convert to Dataframe batch1
df6 = pd.DataFrame(batch6, columns=columns)


### Checkpoint-3

In [143]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     999 non-null    object
 1   industry         1000 non-null   object
 2   website          575 non-null    object
 3   telephone        891 non-null    object
 4   fax              570 non-null    object
 5   address          999 non-null    object
 6   address_maplink  1001 non-null   object
 7   email            872 non-null    object
dtypes: object(8)
memory usage: 62.7+ KB


In [144]:
# save as csv
df6.to_csv('batch6.csv', index = False)

### End of Checkpoint-3

In [145]:
#### Merge the two dataframes
df = pd.concat([df, df6])

### Checkpoint-4

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6006 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     5975 non-null   object
 1   industry         5976 non-null   object
 2   website          3411 non-null   object
 3   telephone        5278 non-null   object
 4   fax              3349 non-null   object
 5   address          5975 non-null   object
 6   address_maplink  6006 non-null   object
 7   email            5187 non-null   object
dtypes: object(8)
memory usage: 422.3+ KB


In [147]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,8c4075d5481d476e945486754f783364@sentry.io
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,info@nicolediem.ch


In [148]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,Max Auer,Vermietung und Verpachtung von eigenen oder ge...,NaN,tel:071 911 21 42,fax:071 911 85 27,Agathafeld 22\n9512 Rossrüti,https://www.google.com/maps/place/Agathafeld%2...,website not accessible
997,Rino Weder AG,Mechanische Werkstätten,http://www.rinoweder.ch/,tel:071 763 60 50,fax:071 763 60 51,Roggenweg 1\n9463 Oberriet SG,https://www.google.com/maps/place/Roggenweg%20...,verkauf@rinoweder.ch
998,Garage Andermatt AG Baar,lnstandhaltung und Reparatur von Automobilen,http://www.garage-andermatt.ch/,tel:041 760 46 46,NaN,Ruessenstrasse 22\n6340 Baar,https://www.google.com/maps/place/Ruessenstras...,info@garage-andermatt.ch
999,"Pizzeria Mare Monte, Ekrem Sinani","Restaurants, Imbissstuben, Tea-Rooms und Gelat...",NaN,tel:071 845 60 06,fax:071 845 60 07,Blumenstrasse 8\n9403 Goldach,https://www.google.com/maps/place/Blumenstrass...,website not accessible
1000,TICINOCOLOR SA,Bewertungen,http://www.ticinocolor.ch/,tel:091 967 79 79,fax:091 967 79 81,via San Gottardo 146\n6942 Savosa,https://www.google.com/maps/place/via%20San%20...,None


In [149]:
df.head()==df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [150]:
df.tail()==df6.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email
996,True,True,False,True,True,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,False,True,True,True,True,True
1000,True,True,True,True,True,True,True,False


### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-7, batch_size = 1000, batch_start_index = 6001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [151]:
# # Extract info of interst
# start_time = time.time()
# batch7_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 6001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [152]:
# # Test-Block !Execute only if running the batch again
# print(len(batch7_initial))
# print(len(batch7_initial[-1]))

In [153]:
# # Test-Block !Execute only if running the batch again
# print(batch7_initial[:3])

### End of Checkpoint-1

In [154]:
batch7df = pd.read_csv('batch7.csv')

In [155]:
batch7df_initial = batch7df.drop(['email'], axis = 1)

In [156]:
batch7df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1000 non-null   object
 1   industry         1001 non-null   object
 2   website          612 non-null    object
 3   telephone        878 non-null    object
 4   fax              562 non-null    object
 5   address          1000 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [157]:
batch7_initial = batch7df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern', 'info@ringler-strahm.ch'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans', '8c4075d5481d476e945486754f783364@sentry.io'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg']]


#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch7 = extract_emails_from_homepage(batch7_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch7)) # should be same as batch_size i.e. 1000
print(len(batch7[-1])) # should be 8
print(batch7[:15]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch7
df7 = pd.DataFrame(batch7, columns=columns)


### Checkpoint-3

In [ ]:
df7.info()

In [ ]:
# save as csv
df7.to_csv('batch7.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df7])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head()==df1.head()

In [ ]:
df.tail()==df7.tail()

### xxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxx

### Batch-8, batch_size = 1000, batch_start_index = 7001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [ ]:
# # Extract info of interst
# start_time = time.time()
# batch8_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 7001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running batch again
# print(len(batch8_initial))
# print(len(batch8_initial[-1]))

In [ ]:
# # Test-Block !Uncomment and execute only if running batch again
# print(batch8_initial[:3])

### End of Checkpoint-1

In [ ]:
batch8df = pd.read_csv('batch8.csv')

In [ ]:
batch8df_initial = batch8df.drop(['email'], axis = 1)

In [ ]:
batch8df_initial.info()

In [ ]:
batch8_initial = batch8df_initial.values.tolist()
print(batch1_initial[:3])

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch8 = extract_emails_from_homepage(batch8_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time

print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch8)) # should be same as batch_size i.e. 1000
print(len(batch8[-1])) # should be 8
print(batch8[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch8
df8 = pd.DataFrame(batch8, columns=columns)


### Checkpoint-3

In [ ]:
df8.info()

In [ ]:
# save as csv
df8.to_csv('batch8.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df8])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df8.tail()

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-9, batch_size = 1000, batch_start_index = 8001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [ ]:
# # Extract info of interst
# start_time = time.time()
# batch9_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 8001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch9_initial))
# print(len(batch9_initial[-1]))

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(batch9_initial[:3])

### End of Checkpoint-1

In [ ]:
batch9df = pd.read_csv('batch9.csv')

In [ ]:
batch9df_initial = batch9df.drop(['email'], axis = 1)

In [ ]:
batch9df_initial.info()

In [ ]:
batch9_initial = batch9df_initial.values.tolist()
print(batch9_initial[:3])

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch9 = extract_emails_from_homepage(batch9_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch9)) # should be same as batch_size i.e. 1000
print(len(batch9[-1])) # should be 8
print(batch9[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch9
df9 = pd.DataFrame(batch9, columns=columns)


### Checkpoint-3

In [ ]:
df9.info()

In [ ]:
# save as csv
df9.to_csv('batch9.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df9])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df9.tail()

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-10, batch_size = 1000, batch_start_index = 9001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [ ]:
# # Extract info of interst
# start_time = time.time()
# batch10_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 9001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch10_initial))
# print(len(batch10_initial[-1]))

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(batch10_initial[:3])

### End of Checkpoint-1

In [ ]:
batch10df = pd.read_csv('batch10.csv')

In [ ]:
batch10df_initial = batch10df.drop(['email'], axis = 1)

In [ ]:
batch10df_initial.info()

In [ ]:
batch10_initial = batch10df_initial.values.tolist()
print(batch10_initial[:3])

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch10 = extract_emails_from_homepage(batch10_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch10)) # should be same as batch_size i.e. 1000
print(len(batch10[-1])) # should be 8
print(batch10[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch10
df10 = pd.DataFrame(batch10, columns=columns)


### Checkpoint-3

In [ ]:
df10.info()

In [ ]:
# save as csv
df10.to_csv('batch10.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df10])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df10.tail()

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxx

### Batch-11, batch_size = 1000, batch_start_index = 10001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [ ]:
# # Extract info of interst
# start_time = time.time()
# batch11_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 10001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch11_initial))
# print(len(batch11_initial[-1]))

In [ ]:
# # # Test-Block !Uncomment and execute only if running the batch again
# print(batch11_initial[:3])

### End of Checkpoint-1

In [ ]:
batch11df = pd.read_csv('batch11.csv')

In [ ]:
batch11df_initial = batch11df.drop(['email'], axis = 1)

In [ ]:
batch11df_initial.info()

In [ ]:
batch11_initial = batch11df_initial.values.tolist()
print(batch11_initial[:3])

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch11 = extract_emails_from_homepage(batch11_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch11)) # should be same as batch_size i.e. 1000
print(len(batch11[-1])) # should be 8
print(batch11[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch11
df11 = pd.DataFrame(batch11, columns=columns)


### Checkpoint-3

In [ ]:
df11.info()

In [ ]:
# save as csv
df11.to_csv('batch11.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df11])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df11.tail()

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-12, batch_size = 1000, batch_start_index = 11001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [ ]:
# # Extract info of interst
# start_time = time.time()
# batch12_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 11001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch12_initial))
# print(len(batch12_initial[-1]))

In [ ]:
# # # Test-Block !Uncomment and execute only if running the batch again
# print(batch12_initial[:3])

### End of Checkpoint-1

In [ ]:
batch12df = pd.read_csv('batch12.csv')

In [ ]:
batch12df_initial = batch12df.drop(['email'], axis = 1)

In [ ]:
batch12df_initial.info()

In [ ]:
batch12_initial = batch12df_initial.values.tolist()


In [ ]:
print(batch12_initial[:3])

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch12 = extract_emails_from_homepage(batch12_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch12)) # should be same as batch_size i.e. 1000
print(len(batch12[-1])) # should be 8
print(batch12[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch11
df12 = pd.DataFrame(batch12, columns=columns)


### Checkpoint-3

In [ ]:
df12.info()

In [ ]:
# save as csv
df12.to_csv('batch12.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df12])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df12.tail()

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxx

### Batch-13, batch_size = 1000, batch_start_index = 12001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [ ]:
# # Extract info of interst
# start_time = time.time()
# batch13_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 12001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch13_initial))
# print(len(batch13_initial[-1]))

In [ ]:
# # # Test-Block !Uncomment and execute only if running the batch again
# print(batch13_initial[:3])

### End of Checkpoint-1

In [ ]:
batch13df = pd.read_csv('batch13.csv')

In [ ]:
batch13df_initial = batch13df.drop(['email'], axis = 1)

In [ ]:
batch13df_initial.info()

In [ ]:
batch13_initial = batch13df_initial.values.tolist()
print(batch13_initial[:3])

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch13 = extract_emails_from_homepage(batch13_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch13)) # should be same as batch_size i.e. 1000
print(len(batch13[-1])) # should be 8
print(batch13[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch11
df13 = pd.DataFrame(batch13, columns=columns)


### Checkpoint-3

In [ ]:
df13.info()

In [ ]:
# save as csv
df13.to_csv('batch13.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df13])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df13.tail()

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxx

### Batch-14, batch_size = 1227, batch_start_index = 13001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [ ]:
# # Extract info of interst
# start_time = time.time()
# batch14_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1227, batch_start_index = 13001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

### Checkpoint-1

In [ ]:
# # Test_Block !Uncomment and execute only if running the batch again
# print(len(batch14_initial))
# print(len(batch14_initial[-1]))

In [ ]:
# # Test_Block !Uncomment and execute only if running the batch again
# print(batch14_initial[:3])

### End of Checkpoint-1

In [ ]:
batch14df = pd.read_csv('batch14.csv')

In [ ]:
batch14df_initial = batch14df.drop(['email'], axis = 1)

In [ ]:
batch14df_initial.info()

In [ ]:
batch14_initial = batch14df_initial.values.tolist()
print(batch14_initial[:3])

#### Following Function Extracts emails from Home Page Only (Takes Less Time: 1 sec per page)

In [ ]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch14 = extract_emails_from_homepage(batch14_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

### Checkpoint-2

In [ ]:
print(len(batch14)) # should be same as batch_size i.e. 1000
print(len(batch14[-1])) # should be 8
print(batch14[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [ ]:
# # Convert to Dataframe batch11
df14 = pd.DataFrame(batch14, columns=columns)


### Checkpoint-3

In [ ]:
df14.info()

In [ ]:
# save as csv
df14.to_csv('batch14.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df14])

### Checkpoing-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df14.tail()

In [ ]:
## Remove duplicates
df = df.drop_duplicates()

In [ ]:
## verify
df.info()

In [ ]:
## Save all batches in a csv file
df.to_csv('all_batches_home.csv', index = False)

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx